<a href="https://colab.research.google.com/github/dandamudi-geeta/Reinforcement-Learning/blob/main/2348512_RL(Lab5).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gym
import numpy as np
from collections import defaultdict

In [ ]:
# Create the Blackjack environment
env = gym.make('Blackjack-v1')

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [ ]:
def generate_episode(env, policy):
    """
    Generate an episode following the given policy.
    Returns a list of (state, action, reward) tuples.
    """
    episode = []
    state = env.reset()
    while True:
        action = policy(state)
        next_state, reward, done, _ = env.step(action)
        episode.append((state, action, reward))
        if done:
            break
        state = next_state
    return episode

In [ ]:
def mc_prediction(policy, env, num_episodes, gamma=1.0):
    """
    Monte Carlo prediction to estimate the state-value function V(s) under the given policy.
    """
    returns_sum = defaultdict(float)
    returns_count = defaultdict(float)
    V = defaultdict(float)

    for _ in range(num_episodes):
        episode = generate_episode(env, policy)
        G = 0  # Initialize the return
        for t in reversed(range(len(episode))):
            state, _, reward = episode[t]
            G = gamma * G + reward
            if state not in [x[0] for x in episode[:t]]:
                returns_sum[state] += G
                returns_count[state] += 1.0
                V[state] = returns_sum[state] / returns_count[state]
    return V

In [31]:
# Define the number of episodes
num_episodes = 5000
epsilon = 0.1

In [58]:
def mc_control_epsilon_greedy(env, num_episodes, gamma=1.0, epsilon=0.1):
    """
    Monte Carlo control using epsilon-greedy policy to find the optimal policy and action-value function Q(s, a).
    """
    Q = defaultdict(lambda: np.zeros(env.action_space.n))
    returns_sum = defaultdict(float)
    returns_count = defaultdict(float)

    def epsilon_greedy_policy(state, epsilon):
        """Epsilon-greedy policy."""
        if np.random.rand() < epsilon:
            return env.action_space.sample()  # Explore
        else:
            return np.argmax(Q[state])  # Exploit

    for _ in range(num_episodes):
        # Generate an episode using the epsilon-greedy policy
        episode = []
        state = env.reset()
        while True:
            action = epsilon_greedy_policy(state, epsilon)
            next_state, reward, done, _ = env.step(action)
            episode.append((state, action, reward))
            if done:
                break
            state = next_state

        G = 0  # Initialize the return
        for t in reversed(range(len(episode))):
            state, action, reward = episode[t]
            G = gamma * G + reward
            if (state, action) not in [(x[0], x[1]) for x in episode[:t]]:
                returns_sum[(state, action)] += G
                returns_count[(state, action)] += 1.0
                Q[state][action] = returns_sum[(state, action)] / returns_count[(state, action)]

    # Derive the optimal policy from Q
    optimal_policy = {state: np.argmax(action_values) for state, action_values in Q.items()}
    return optimal_policy, Q  # Ensure this return is properly indented


In [59]:
# Run Monte Carlo Control
optimal_policy, Q = mc_control_epsilon_greedy(env, num_episodes, epsilon=epsilon)

/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:241: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


In [60]:
# Display the Optimal Policy
print("\nOptimal Policy Derived from Q:")
for state, action in list(optimal_policy.items())[:10]:  # Display a subset of the results
    print(f"State: {state}, Action: {action}")


Optimal Policy Derived from Q:
State: (21, 10, True), Action: 0
State: (11, 8, False), Action: 1
State: (20, 2, False), Action: 0
State: (18, 5, False), Action: 0
State: (16, 9, False), Action: 0
State: (14, 10, False), Action: 1
State: (20, 1, False), Action: 0
State: (17, 10, True), Action: 1
State: (20, 10, False), Action: 0
State: (12, 2, False), Action: 1
